In [77]:
from random import randint
import torch
import numpy as np

Carbin_Count = 9  # 


#定义环境
class Environment:
    def __init__(self):
        self.state = None
        self.time_spend = 0  # 使用的总共时间 要惩罚时间的
        self.reset()

    def reset(self):
        # 每个船舱的重量
        self.time_spend = 0  # 使用的总共时间 要惩罚时间的
        self.state = torch.FloatTensor(np.random.uniform(1, 2, (1, Carbin_Count)))
        return self.state

    def is_over(self, action: torch.Tensor = []):
        def is_fault(action: torch.Tensor):
            last_action = -1
            for i in action:
                index = int(i)  # 映射到货轮
                if index == 0:
                    continue
                if index < last_action:
                    return True
                last_action = index
            return False

        if is_fault(action):
            return -1
        # 判断state每一项是否小于0
        if torch.all(self.state < 0):
            return 1
        return 0

    def get_state_reword(self, action: torch.Tensor):
        # 当前状态的分数,比如为负数了还有人来卸载就应该扣分
        reword = 0
        for i in action:
            index = int(i)  # 映射到货轮
            if index == 0:
                reword -= 3
                continue
            # 动起来的奖励
            reword += 1 * 5
            if self.state[0][index - 1] < 0:
                reword += 5 * (self.state[0][index - 1] * 13)
        is_over = self.is_over(action)
        if is_over == 1:
            reword += 1200
        elif is_over == -1:
            reword -= 200
        return reword

    def step(self, action):
        for i in action:
            index = int(i)  # 映射到货轮
            if index == 0:
                continue
            self.state[0][index - 1] -= 0.1
        self.time_spend += 1
        reword = self.get_state_reword(action)
        return self.state, reword, self.is_over(action) != 0, None


class MyWrapper:
    N = 3  # 港机数量

    def __init__(self):
        self.env = Environment()
        self.step_n = 0

    def reset(self):
        state = self.env.reset()
        self.step_n = 0
        return state

    def step(self, action):
        # print(action)
        state, reward, terminated, info = self.env.step(action)
        over = terminated
        #限制最大步数
        self.step_n += 1
        if self.step_n > 200:
            over = True
        return state, reward, over

    def to_show_state(self):
        return '%s  步骤:%s  游戏是否结束:%s' % (str(self.env.state), str(self.env.time_spend), str(self.env.is_over()))


env = MyWrapper()

env.reset()
env.step(torch.full((1, 1), 0.5, dtype=torch.float32))
env.to_show_state()

'tensor([[1.3996, 1.9350, 1.5702, 1.3020, 1.1738, 1.0413, 1.0137, 1.6923, 1.3770]])  步骤:1  游戏是否结束:0'

In [78]:
import torch


class A2C:

    def __init__(self, model_actor, model_critic, model_critic_delay,
                 optimizer_actor, optimizer_critic):
        self.model_actor = model_actor
        self.model_critic = model_critic
        self.model_critic_delay = model_critic_delay
        self.optimizer_actor = optimizer_actor
        self.optimizer_critic = optimizer_critic

        self.model_critic_delay.load_state_dict(self.model_critic.state_dict())
        self.requires_grad(self.model_critic_delay, False)

    def soft_update(self, _from, _to):
        for _from, _to in zip(_from.parameters(), _to.parameters()):
            value = _to.data * 0.99 + _from.data * 0.01
            _to.data.copy_(value)

    def requires_grad(self, model, value):
        for param in model.parameters():
            param.requires_grad_(value)

    def train_critic(self, state, reward, next_state, over):
        self.requires_grad(self.model_critic, True)
        self.requires_grad(self.model_actor, False)

        #计算values和targets
        value = self.model_critic(state)

        with torch.no_grad():
            target = self.model_critic_delay(next_state)
        target = target * 0.99 * (1 - over) + reward
        # print('xxxx', value.size(), target.size(), reward.size())
        #时序差分误差,也就是tdloss
        loss = torch.nn.functional.mse_loss(value, target)

        loss.backward()
        self.optimizer_critic.step()
        self.optimizer_critic.zero_grad()
        self.soft_update(self.model_critic, self.model_critic_delay)

        #减去value相当于去基线
        return (target - value).detach()

    def train_actor(self, state, action, value):
        self.requires_grad(self.model_critic, False)
        self.requires_grad(self.model_actor, True)

        #重新计算动作的概率
        prob = self.model_actor(state)
        prob = prob.gather(dim=1, index=action)

        #根据策略梯度算法的导函数实现
        #函数中的Q(state,action),这里使用critic模型估算
        prob = (prob + 1e-8).log() * value
        loss = -prob.mean()

        loss.backward()
        self.optimizer_actor.step()
        self.optimizer_actor.zero_grad()

        return loss.item()


model_actor = [
    torch.nn.Sequential(
        torch.nn.Linear(9, 64),
        torch.nn.ReLU(),
        torch.nn.Linear(64, 64),
        torch.nn.ReLU(),
        torch.nn.Linear(64, Carbin_Count + 1),
        torch.nn.Softmax(dim=1),
    ) for _ in range(env.N)
]

model_critic, model_critic_delay = [
    torch.nn.Sequential(
        torch.nn.Linear(9, 64),
        torch.nn.ReLU(),
        torch.nn.Linear(64, 64),
        torch.nn.ReLU(),
        torch.nn.Linear(64, 1),
    ) for _ in range(2)
]

optimizer_actor = [
    torch.optim.Adam(model_actor[i].parameters(), lr=1e-3)
    for i in range(env.N)
]
optimizer_critic = torch.optim.Adam(model_critic.parameters(), lr=5e-3)

a2c = [
    A2C(model_actor[i], model_critic, model_critic_delay, optimizer_actor[i],
        optimizer_critic) for i in range(env.N)
]
# x = torch.FloatTensor([1,2,3,4,6,7,8,8,4]).resize(1,Carbin_Count)
# print(model_actor[0](x))
model_actor = None
model_critic = None
model_critic_delay = None
optimizer_actor = None
optimizer_critic = None

a2c


In [79]:

import random


#玩一局游戏并记录数据
def play(show=False):
    state = []
    action = []
    reward = []
    next_state = []
    over = []

    s = env.reset()
    o = False
    while not o:
        a = []
        for i in range(env.N):
            #计算动作
            prob = a2c[i].model_actor(torch.FloatTensor(s).reshape(
                1, -1))[0].tolist()
            # print(s, prob)
            a.append(random.choices(range(Carbin_Count + 1), weights=prob, k=1)[0])

        #执行动作
        ns, r, o = env.step(a)

        state.append(s)
        action.append(a)
        reward.append(r)
        next_state.append(ns)
        over.append(o)

        s = ns

        if show:
            print(a, env.to_show_state())
    # print(state[0])
    # print(type(state), len(state))
    state = torch.tensor([item.numpy() for item in state])
    action = torch.LongTensor(action).unsqueeze(-1)
    reward = torch.FloatTensor(reward).unsqueeze(-1).unsqueeze(-1)
    next_state = torch.tensor([item.numpy() for item in next_state])
    over = torch.LongTensor(over).reshape(-1, 1)

    return state, action, reward, next_state, over, reward.sum().item()


state, action, reward, next_state, over, reward_sum = play()

reward_sum, state.size(), reward.size(), action.size()

(-193.0, torch.Size([1, 1, 9]), torch.Size([1, 1, 1]), torch.Size([1, 3, 1]))

In [80]:
def train():
    #训练N局
    for epoch in range(3_000):
        state, action, reward, next_state, over, _ = play()

        #合并部分字段
        state_c = state.flatten(start_dim=1)
        reward_c = reward.sum(dim=1)
        next_state_c = next_state.flatten(start_dim=1)

        for i in range(env.N):
            value = a2c[i].train_critic(state_c, reward_c, next_state_c, over)
            loss = a2c[i].train_actor(state_c, action[:, i], value)

        if epoch % 250 == 0:
            test_result = sum([play()[-1] for _ in range(20)]) / 20
            print(epoch, loss, test_result)


train()

0 -477.1976623535156 -181.75
250 -5.270344257354736 -151.05
500 -1.5087164640426636 -572.780606842041
750 0.028594359755516052 -215.3936586380005
1000 -21.804866790771484 -210.26653938293458
1250 40.616859436035156 -1357.6139625549317
1500 1.8938491344451904 866.8003646850586
1750 -3322.0166015625 -4423.916413116455
2000 24.359785079956055 1177.1616806030274
2250 -24.82537078857422 1206.2004661560059
2500 3.3432013988494873 1600.9697021484376
2750 -5.417761325836182 1616.6085876464845


In [82]:
play(True)[-1]

[1, 4, 9] tensor([[1.5364, 1.3467, 1.0137, 1.5024, 1.8272, 1.6970, 1.9312, 1.1900, 1.6986]])  步骤:1  游戏是否结束:0
[1, 5, 9] tensor([[1.4364, 1.3467, 1.0137, 1.5024, 1.7272, 1.6970, 1.9312, 1.1900, 1.5986]])  步骤:2  游戏是否结束:0
[1, 6, 9] tensor([[1.3364, 1.3467, 1.0137, 1.5024, 1.7272, 1.5970, 1.9312, 1.1900, 1.4986]])  步骤:3  游戏是否结束:0
[1, 6, 9] tensor([[1.2364, 1.3467, 1.0137, 1.5024, 1.7272, 1.4970, 1.9312, 1.1900, 1.3986]])  步骤:4  游戏是否结束:0
[1, 4, 9] tensor([[1.1364, 1.3467, 1.0137, 1.4024, 1.7272, 1.4970, 1.9312, 1.1900, 1.2986]])  步骤:5  游戏是否结束:0
[1, 5, 9] tensor([[1.0364, 1.3467, 1.0137, 1.4024, 1.6272, 1.4970, 1.9312, 1.1900, 1.1986]])  步骤:6  游戏是否结束:0
[1, 4, 9] tensor([[0.9364, 1.3467, 1.0137, 1.3024, 1.6272, 1.4970, 1.9312, 1.1900, 1.0986]])  步骤:7  游戏是否结束:0
[1, 6, 9] tensor([[0.8364, 1.3467, 1.0137, 1.3024, 1.6272, 1.3970, 1.9312, 1.1900, 0.9986]])  步骤:8  游戏是否结束:0
[1, 4, 9] tensor([[0.7364, 1.3467, 1.0137, 1.2024, 1.6272, 1.3970, 1.9312, 1.1900, 0.8986]])  步骤:9  游戏是否结束:0
[1, 5, 9] tensor([[

1578.3857421875